# Honeybee Daylight From gbXML

Honeybee is a python building energy and daylighting simulation library mainly used through the Rhino/Grasshopper GUI. The core components and functionality of this library however do not require any specific external geometry package to work. The series of tutorials presented in this repository will aim to demonstrate that a series of complex and useful simulations can be (and maybe even should be) carried out outwith of the Rhino/Grasshopper ecosystem to facilitate reproductibility, version control, speed and scale.

Before any simulation is done however we will first need a model. A common file format for Environmental Buidling Simulations is gbXML, which is why we will be using it as the basis of our Honeybee model generation. In this tutorial we will carry out the following tasks:

1. Download and parse a gbXML file
2. Extract all surfaces/windows from the parsed gbXML
3. Extract all zones and generate simulation grids
4. Save grids and surfaces in JSON format for later use

## 1. Getting a gbXML
The ladybug tools project has introduced a gbXML rendering bug called [`spider`](https://www.ladybug.tools/spider/#README.md) which helps visualise and debug such files. As such it seems fitting to use on the main examples from this project and apply it to this tutorial.

We will be using the [`bristol-clifton-downs` example file](https://www.ladybug.tools/spider/gbxml-viewer/r12/gv-app/gv-app.html#https://rawgit.com/ladybug-tools/spider/master/gbxml-sample-files/bristol-clifton-downs-fixed.xml). 

We download and read the file from the url indicated in the code snippet below. We also write the file to our local folder to inspect elemets of it in our code editor.

In [1]:
from urllib2 import urlopen

gbxml_address = 'https://rawgit.com/ladybug-tools/spider/master/gbxml-sample-files/bristol-clifton-downs-fixed.xml'

request = urlopen(gbxml_address)

gbxml_string = request.read()

with open('data/bristol-clifton-downs-fixed.xml', 'w') as f:
    f.write(gbxml_string)

Next we use the [`minidom`](https://docs.python.org/2/library/xml.dom.minidom.html) package to parse the gbXML file and split it into two main objects:
* `gbxml_surfaces`: contains all the surfaces and subsurfaces in the gbXML file. We will retrieve geometry, type, nam and possibly material information from this object.
* `gbxml_spaces`: contains all the space information in the gbXML. We are mostly interested in the name, space polygon and occupancy information.

In [2]:
from xml.dom import minidom

gbxml = minidom.parseString(gbxml_string)
gbxml_surfaces = gbxml.getElementsByTagName('Surface')
gbxml_spaces = gbxml.getElementsByTagName('Space')


## 2. gbXML to Honeybee Surfaces

The `gbxml_surfaces` object will be parsed in order to produce `Honeybee Surface` objects. These objects will contain the geometry, name, type and material of each surface in the original gbXML. To understand how we will parse these objects lets first look at how a surface is represented in the gbXML file. This is shown in the cell below:


```xml
<Surface surfaceType="ExteriorWall" exposedToSun="true" id="aim6050">
  <AdjacentSpaceId spaceIdRef="aim0682"/>
  <RectangularGeometry id="aim6051">
    <Azimuth>93.4364318847656</Azimuth>
    <CartesianPoint>
      <Coordinate>500.1</Coordinate>
      <Coordinate>1022.647</Coordinate>
      <Coordinate>19.25</Coordinate>
    </CartesianPoint>
    <Tilt>90</Tilt>
    <Width>3.86800003051758</Width>
    <Height>3.90000009536743</Height>
  </RectangularGeometry>
  <PlanarGeometry>
    <PolyLoop>
      <CartesianPoint>
        <Coordinate>500.1</Coordinate>
        <Coordinate>1022.647</Coordinate>
        <Coordinate>19.25</Coordinate>
      </CartesianPoint>
      <CartesianPoint>
        <Coordinate>500.3318</Coordinate>
        <Coordinate>1026.508</Coordinate>
        <Coordinate>19.25</Coordinate>
      </CartesianPoint>
      <CartesianPoint>
        <Coordinate>500.3318</Coordinate>
        <Coordinate>1026.508</Coordinate>
        <Coordinate>23.15</Coordinate>
      </CartesianPoint>
      <CartesianPoint>
        <Coordinate>500.1</Coordinate>
        <Coordinate>1022.647</Coordinate>
        <Coordinate>23.15</Coordinate>
      </CartesianPoint>
    </PolyLoop>
  </PlanarGeometry>
  <Opening openingType="OperableWindow" id="aim6071">
    <RectangularGeometry id="aim6072">
      <CartesianPoint>
        <Coordinate>0</Coordinate>
        <Coordinate>0.05</Coordinate>
        <Coordinate>0</Coordinate>
      </CartesianPoint>
      <Width>1.5780000015062</Width>
      <Height>3</Height>
    </RectangularGeometry>
    <PlanarGeometry>
      <PolyLoop>
        <CartesianPoint>
          <Coordinate>500.1</Coordinate>
          <Coordinate>1022.647</Coordinate>
          <Coordinate>19.3</Coordinate>
        </CartesianPoint>
        <CartesianPoint>
          <Coordinate>500.1946</Coordinate>
          <Coordinate>1024.222</Coordinate>
          <Coordinate>19.3</Coordinate>
        </CartesianPoint>
        <CartesianPoint>
          <Coordinate>500.1946</Coordinate>
          <Coordinate>1024.222</Coordinate>
          <Coordinate>22.3</Coordinate>
        </CartesianPoint>
        <CartesianPoint>
          <Coordinate>500.1</Coordinate>
          <Coordinate>1022.647</Coordinate>
          <Coordinate>22.3</Coordinate>
        </CartesianPoint>
      </PolyLoop>
    </PlanarGeometry>
    <CADObjectId>WinInst: SIM_EXT_GLZ [898655]</CADObjectId>
    <Name>N-00_109-E-W-52-W-2</Name>
  </Opening>
  <CADObjectId>Basic Wall: SIM_EXT_SLD [882893]</CADObjectId>
  <Name>N-00_109-E-W-52</Name>
</Surface>
```

Quite the mouthful right?! Yeah XML is not the most human friendly file format. What you will notice however is that:
* The surface type is contained in the first element of the surface xml object under the name `surfaceType`
* A list of points composing the planar surface geometry is available under `PlanarGeometry` -> `PolyLoop`
* The name of the surface can be found at the very bottom under `Name`
* In this case the `CADObjectId` tag seems to contain information about the specific construction type used.
* Openings such as windows or doors belonging to the surface are contained under `Opening`. These objects in turn contain surface types, names and geometry information

A parser to retrieve `name`, `surface_type` and `surface_points` has been written and is saved in `lib/gbxml_surface_parser` for brevity's sake. Feel free to take a peek at the functions and how they work if you are so inclined.

The code below demonstrates how this information is extracted from each `gbxml_surface` object and turned into a `honeybee_surface`:

In [3]:
from lib.gbxml_surface_parser import get_clean_name, get_surface_type, get_points, get_fenestration

from honeybee.surfacetype import AirWall, Context
from honeybee.hbsurface import HBSurface
from honeybee.hbshadesurface import HBShadingSurface

hb_surfaces = []
hb_context_surfaces = []


for surface in gbxml_surfaces:
    # Get surface name
    surface_name_raw = surface.getElementsByTagName('Name')[0].childNodes[0].nodeValue
    surface_name = get_clean_name(surface_name_raw)
    
    # Get surface material information
    material_name = surface.getElementsByTagName('CADObjectId')[0].childNodes[0].nodeValue

    # Get surface type and return a compatible Honeybee Surface Type class
    surface_type = get_surface_type(surface)
    
    # Get all the points in the Planar Geometry Polyloop and return them as a list of (x, y, z)
    surface_points = get_points(surface)

    # Ignore AirWalls => They are not important for daylighting
    if isinstance(surface_type, AirWall):
        pass

    # Sometimes air walls are only indicated through materials...
    elif '_AIR' in material_name:
        pass
    
    # Any context surfaces are saved seperately to be added or removed depending on the use case
    elif isinstance(surface_type, Context):
        hb_surface = HBShadingSurface(name=surface_name,
                                      sorted_points=surface_points,
                                      is_name_set_by_user=True)

        hb_context_surfaces.append(hb_surface)

    # Finally we get to Honeybee Surfaces!
    else:
        hb_surface = HBSurface(name=surface_name,
                               sorted_points=surface_points,
                               surface_type=surface_type,
                               is_name_set_by_user=True)

        # This function does essentially the same task as the loop above but for glazing
        fenestrations = get_fenestration(surface)
        for fenestration in fenestrations:
            hb_surface.add_fenestration_surface(fenestration)
            
        hb_surfaces.append(hb_surface)
    
print("{} Honeybee Surfaces parsed".format(len(hb_surfaces)))
print("{} Context Surfaces parsed".format(len(hb_context_surfaces)))

Path to OpenStudio is set to: C:/Program Files/OpenStudio 1.14.0
Path to radiance is set to: c:/radiance
Path to perl is set to: C:/Program Files/OpenStudio 1.14.0\strawberry-perl-5.16.2.1-32bit-portable-reduced
442 Honeybee Surfaces parsed
75 Context Surfaces parsed


For convenience this section of code has been reduced to a single function which can be reused once the user knows what it does:

In [5]:
from lib.gbxml_surface_parser import get_hb_surfaces

hb_surfaces, hb_context_surfaces = get_hb_surfaces(gbxml_surfaces)

print("{} Honeybee Surfaces parsed".format(len(hb_surfaces)))
print("{} Context Surfaces parsed".format(len(hb_context_surfaces)))

442 Honeybee Surfaces parsed
75 Context Surfaces parsed


## Parse Zones into Analysis Grids

In [10]:
from lib.gbxml_surface_parser import get_points, get_clean_name
from lib.gbxml_room_grid_parser import grid_from_floor, points_in_floor_polygon
from honeybee.radiance.analysisgrid import AnalysisGrid

analysis_grids = []

# Check space 'conditionType' to be 'HeatedAndCooled', if 'unconditioned' then do not use.

for space in gbxml_spaces:
    space_name_raw = space.getElementsByTagName('Name')[0].childNodes[0].nodeValue
    name = get_clean_name(space_name_raw)
    floor_points = get_points(space)
    grid = grid_from_floor(floor_points)
    
    analysis_grid = AnalysisGrid.from_points_and_vectors(grid, name=name)

    analysis_grids.append(analysis_grid)

len(analysis_grids)

87

### IEQc: 8.1 Daylight and Views – Daylight
Demonstrate through computer simulation that the applicable spaces achieve daylight
illuminance levels of a minimum of 10 footcandles (fc) (110 Lux) and a maximum of 500 fc
(5,400 Lux) in a clear sky condition on September 21 at 9 a.m. and 3 p.m.; areas
Provide glare control devices to avoid high-contrast situations that could impede visual
tasks. However, designs that incorporate view-preserving automated shades for glare
control may demonstrate compliance for only the minimum 10 fc (110 lux) illuminance level. 

In [11]:
from honeybee.radiance.recipe.pointintime.gridbased import GridBased as PiTGridBased
from honeybee.radiance.sky.climatebased import ClimateBased as ClimateBasedSky
from ladybug.wea import Wea

epw_file_path = 'data/GBR_Birmingham.035340_IWEC.epw'

wea = Wea.from_epw_file(epw_file_path)

sky_1 = ClimateBasedSky.from_wea(wea=wea, month=9, day=21, hour=9, north=0)

sky_2 = ClimateBasedSky.from_wea(wea=wea, month=9, day=21, hour=15, north=0)

In [12]:
# from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as DFGridBased
# import os

# df_recipe = DFGridBased(analysis_grids=analysis_grids, hb_objects=hb_surfaces)


# target_folder = os.path.join(os.getcwd(), 'recipes')

# command_path = df_recipe.write(target_folder, 'test')

# df_recipe.run(command_path)

In [13]:
analysis_grids = df_recipe.results()

analysis_grids[0].to_json()

NameError: name 'df_recipe' is not defined

In [14]:
analysis_grid = analysis_grids[10]

recipe_1 = PiTGridBased(sky=sky_1, analysis_grids=[analysis_grid], hb_objects=hb_surfaces)

recipe_2 = PiTGridBased(sky=sky_2, analysis_grids=[analysis_grid], hb_objects=hb_surfaces)


Found 517 opaque surfaces.
Found 268 fenestration surfaces.
Found 0 window-groups.
Found 517 opaque surfaces.
Found 268 fenestration surfaces.
Found 0 window-groups.


In [15]:
import os

target_folder = os.path.join(os.getcwd(), 'recipes')

command_path = recipe_1.write(target_folder, 'test')
recipe_1.run(command_path)

Writing recipe contents to: C:\Users\Antoine\Documents\projects\honeybee tutorials\recipes\test\gridbased


True